In [ ]:
import sys
sys.path.append('../../../')
from fastai.imports import *
from fastai.dataset import *

In [17]:
class SaveFeatures():
    features=None
    def __init__(self, m): self.hook = m.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output): self.features = output
    def close(self): self.hook.remove()
        
def gram(input):
    b,c,h,w = input.size()
    x = input.view(b*c, -1)
    return torch.mm(x, x.t())/input.numel()*1e6

def get_gram(filename):
    img = open_image(filename)
    img_tfms = val_tfms(img)
    m_vgg(VV(img_tfms[None]))
    img_act = act.features.clone()
    return to_np(gram(img_act).view(-1))

m_vgg = to_gpu(vgg16(True)).eval()
set_trainable(m_vgg, False)
sz=288
_, val_tfms = tfms_from_model(vgg16, sz)
act = SaveFeatures(children(m_vgg)[5])